In [24]:

from langchain_huggingface import HuggingFaceEndpoint

#Text wrapper
import textwrap
def print_text(llm_response):
    wrapper=textwrap.TextWrapper(width=104)
    string=wrapper.fill(text=llm_response)
    return print(string)


import torch
device="cuda" if torch.cuda.is_available() else "cpu"

In [10]:
from pypdf import PdfReader

reader = PdfReader("microsoft_annual_report_2022.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(word_wrap(pdf_texts[0]))

1 Dear shareholders, colleagues, customers, and partners:  
We are
living through a period of historic economic, societal, and
geopolitical change. The world in 2022 looks nothing like 
the world in
2019. As I write this, inflation is at a 40 -year high, supply chains
are stretched, and the war in Ukraine is 
ongoing. At the same time, we
are entering a technological era with the potential to power awesome
advancements 
across every sector of our economy and society. As the
world’s largest software company, this places us at a historic

intersection of opportunity and responsibility to the world around us.
 
Our mission to empower every person and every organization on the
planet to achieve more has never been more 
urgent or more necessary.
For all the uncertainty in the world, one thing is clear: People and
organizations in every 
industry are increasingly looking to digital
technology to overcome today’s challenges and emerge stronger. And no

company is better positioned to help th

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
character_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n",'\n','.',' ',''],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts=character_splitter.split_text(\
    '\n\n'.join(pdf_texts))


In [15]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

c:\ProgramData\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


increased, due in large part to significant global datacenter
expansions and the growth in xbox sales and usage. despite these
increases, we remain dedicated to achieving a net - zero future. we
recognize that progress won ’ t always be linear, and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time. on the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate over 1. 3 million cubic meters of volumetric benefits in nine
water basins around the world. progress toward our zero waste
commitment included diverting more than 15, 200 metric tons of solid
waste otherwise headed to landfills and incinerators, as well as
launching new circular centers to increase reuse and reduce e - waste
at our datacenters. we contracted to protect over 17, 000 acres of land
( 50 % more than the land we use to operate ), thus achieving our

Total chunks: 349


In [40]:
import chromadb

# 1. Initialize the embedding function and define the ids
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
embedding_function=SentenceTransformerEmbeddingFunction()

# Ids of the documents
ids=[str(i) for i in range(len(token_split_texts))]


In [41]:
# 2. be the client of chroma db and creat a collection
chroma_client=chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report",\
                                                    embedding_function=embedding_function)


# 3. add ids and 
chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

: 

In [ ]:
query="What was the total revenue?"
